### Dataset con informacion financiera, gastos y entradas de personas de distinto sexo y distritos o barrios o sectores de una ciudad, estas ultimas son variables categoricas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv("../datasets/ecom-expense/Ecom Expense.csv")

In [ ]:
df.head()

In [ ]:
#creamos un dummy para las variables categoricas, con esto se asignan variables numericas a cada categoria
dummy_gender = pd.get_dummies(df["Gender"],prefix ="Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"],prefix ="City")


In [ ]:
#quedan 2 columnas para la variable categorica Gender
dummy_gender.head()

In [ ]:
#quedan 3 columnas para la variable categorica City Tier
dummy_city_tier.head()

In [ ]:
#Se unen estas variables dummy al dataframe principal con JOIN
column_names = df.columns.values.tolist() #se crea una lista con los nombres de columnas del df principal
column_names

In [ ]:
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new.head()

In [ ]:
df_new = df_new[column_names].join(dummy_city_tier)
df_new.head()


In [ ]:
#ahora ver como incluimos las variables dummy en el modelo lineal que buscamos crear para predecir el gasto total
feature_cols = ["Monthly Income","Transaction Time",
                "Gender_Female","Gender_Male",
                "City_Tier 1","City_Tier 2","City_Tier 3",
               "Record"]

In [ ]:
#variables a usar como predictores
X = df_new[feature_cols]
#variable Y que se quiere predecir
Y = df_new["Total Spend"]

In [ ]:
#ahora se crea el modelo lineal con sklearn
lm = LinearRegression()
lm.fit(X,Y)

In [ ]:
print(lm.intercept_)
print(lm.coef_)

In [ ]:
#se unen las columnas con con sus respectivos coeficientes
list(zip(feature_cols,lm.coef_))

In [ ]:
lm.score(X,Y) #R2 sale bajo 0.19, hay que refinar el modelo, agregaremos la variable Record, ver como aumenta mucho el R2

### El modelo entonces puede ser escrito como
Total_Spend = -79.4171303013718 +'Monthly Income'*0.1475389804920574  + 'Transaction Time'*0.15494612549589393 +             
                                 'Gender_Female' *-131.02501325554675 + 'Gender_Male'*131.02501325554664       +
                                 'City_Tier 1'   *76.764326010495     + 'City_Tier 2'*55.13897430923278        +
                                 'City_Tier 3'   *-131.90330031972783 +  'Record'    *772.2334457445645
### Lo anterior es la forma global del modelo, pero se debe dividir segun las variables categoricas
    * Si es hombre y vive en CT1

In [ ]:
# calculamos la desviacion tipica de los residuos RSE
df_new["prediction"] = -79.4171303013718 +df_new['Monthly Income']*0.1475389804920574  + df_new['Transaction Time']*0.15494612549589393+ df_new['Gender_Female']*(-131.02501325554675) + df_new['Gender_Male']*131.02501325554664 +  df_new['City_Tier 1']*76.764326010495 + df_new['City_Tier 2']*55.13897430923278+  df_new['City_Tier 3']*(-131.90330031972783) + df_new[ 'Record']*772.2334457445645                                                     

In [ ]:
df_new.head()

In [ ]:
SSD = np.sum((df_new["prediction"] - df_new["Total Spend"])**2)

In [ ]:
SSD

In [ ]:
RSE = np.sqrt(SSD/(len(df_new)-len(feature_cols)-1))

In [ ]:
RSE

In [ ]:
sales_mean =np.mean(df_new["Total Spend"])
sales_mean

In [ ]:
error = RSE/sales_mean


In [ ]:
error

### Eliminar variables dummies redundantes

In [ ]:
dummy_gender = pd.get_dummies(df["Gender"],prefix="Gender").iloc[:,1:]
dummy_gender.head()

In [ ]:
dummy_city_tier = pd.get_dummies(df["City Tier"],prefix="City").iloc[:,1:]
dummy_city_tier.head()


In [ ]:
column_names = df.columns.values.tolist()
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new = df_new[column_names].join(dummy_city_tier)
df_new.head()

In [ ]:
feature_cols = ["Monthly Income","Transaction Time",
                "Gender_Male",
               "City_Tier 2","City_Tier 3",
               "Record"]
X = df_new[feature_cols]
Y = df_new["Total Spend"]
lm = LinearRegression()
lm.fit(X,Y)

In [ ]:
print(lm.intercept_)

In [ ]:
list(zip(feature_cols,lm.coef_))

In [ ]:
#el modelo noha cambiado, solo lo hemos simplificado con menos variables dummies
lm.score(X,Y)

### Comparando los coeficientes de ambos implementaciones del modelo
#### Con todas las variables del modelo
* ('Monthly Income', 0.1475389804920574),
* ('Transaction Time', 0.15494612549589393),
* ('Gender_Female', -131.02501325554675),
*  ('Gender_Male', 131.02501325554664),
*  ('City_Tier 1', 76.764326010495),
*  ('City_Tier 2', 55.13897430923278),
*  ('City_Tier 3', -131.90330031972783),
*  ('Record', 772.2334457445645)]

#### luego de enmascarar las variables dummies
* ('Monthly Income', 0.14753898049205744),
* ('Transaction Time', 0.15494612549589837),
* ('Gender_Male', 262.0500265110939),
* ('City_Tier 2', -21.62535170126288),
* ('City_Tier 3', -208.66762633022293),
* ('Record', 772.2334457445637)]

#### Los cambios se reflejan al quitar el caso base de las variables que hemos elimnado, el modelo queda igual, pero los coeficientes se redistribuyen
* Gender Male
    * antes -> 131.02501325554664
    * despues -> 262.0500265110939 = 131.02501325554675 -(-131.02501325554675)
* Gender Female
    * antes-> -131.02501325554675)
    * despues-> 0 (recordar que lo hemos quitado)
* CT1
    * antes  -> 76.764326010495
    * despues-> 0 (recordar que lo hemos quitado)
* CT2
    * antes ->  55.13897430923278
    * despues ->  -21.62535170126288 = 55.13897430923278 - 76.764326010495
* CT3
    * antes  -> -131.90330031972783
    * despues -> -208.66762633022293 = -131.90330031972783 -76.764326010495

### Transformacion de variables con conseguir relacion no lineal